# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,-3.71,100,100,8.75,CA,1706557211
1,1,shimoda,34.6667,138.9500,3.95,69,3,4.32,JP,1706557211
2,2,lehliu-gara,44.4333,26.8500,2.01,80,23,4.77,RO,1706557212
3,3,mount gambier,-37.8333,140.7667,11.25,93,8,2.35,AU,1706557212
4,4,edinburgh of the seven seas,-37.0676,-12.3116,16.59,71,41,9.34,SH,1706557212


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size="Humidity",
    color="City"
)

# Display the map
map_plot_1



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
new_city_data = city_data_df.loc[((city_data_df[("Max Temp")] <28) & (city_data_df["Max Temp"] > 21)) & 
                                 ((city_data_df["Cloudiness"]<10))
                                 &(city_data_df["Wind Speed"]<5)]
# new_city_data.count()

# Drop any rows with null values
# new_city_data.dropna(how='any')
# new_city_data.count()

# Display sample data
new_city_data


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,beira,-19.8436,34.8389,26.05,83,0,3.60,MZ,1706557217
77,77,timbuktu,16.7735,-3.0074,23.35,12,0,4.67,ML,1706556968
144,144,salalah,17.0151,54.0924,25.05,69,0,2.06,OM,1706556980
146,146,shani,10.2162,12.0620,26.39,16,1,2.91,NG,1706557239
181,181,stephenville,32.2207,-98.2023,21.77,32,0,3.60,US,1706557172
239,239,el granada,37.5027,-122.4694,23.10,70,0,3.60,US,1706557251
261,261,dhidhdhoo,6.8833,73.1000,27.57,66,6,3.78,MV,1706557254
298,298,lompoc,34.6391,-120.4579,25.56,42,0,3.09,US,1706557259
382,382,kontcha,7.9667,12.2333,23.96,20,6,0.73,CM,1706557273
414,414,atar,20.5169,-13.0499,21.51,11,0,3.82,MR,1706557020


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city_df = new_city_data.copy()
city_df = city_df[["City","Country","Lat","Lng","Humidity"]]
city_df
hotel_df=pd.DataFrame(city_df)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
34,beira,MZ,-19.8436,34.8389,83,
77,timbuktu,ML,16.7735,-3.0074,12,
144,salalah,OM,17.0151,54.0924,69,
146,shani,NG,10.2162,12.0620,16,
181,stephenville,US,32.2207,-98.2023,32,
239,el granada,US,37.5027,-122.4694,70,
261,dhidhdhoo,MV,6.8833,73.1000,66,
298,lompoc,US,34.6391,-120.4579,42,
382,kontcha,CM,7.9667,12.2333,20,
414,atar,MR,20.5169,-13.0499,11,


In [12]:
# citylat=hotel_df["Lat"]
# citylat
# for index,row in hotel_df.iterrows():
#     print(hotel_df["Lat"])
#     print(row["Lat"])
#     print(index)

In [24]:
# Set parameters to search for a hotel
radius = 10000
categories="accommodation.hotel"
limit=1
latitude=-19.8436
longitude=34.8389
city="beira"
filters = f"circle:{longitude},{latitude},{radius}"
params = {"categories":categories,
    "limit":limit,
    "filter":filters,
#      "bias":bias,
    "radius":radius,
    "apiKey":geoapify_key  
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

    
    # Set base URL
base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
name_address = requests.get(base_url,params=params).json()
name_address    

Starting hotel search


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Hotel Savoy',
    'country': 'Mozambique',
    'country_code': 'mz',
    'region': 'Zona Centro',
    'state': 'Sofala Province',
    'county': 'Cidade da Beira',
    'city': 'Beira',
    'postcode': '2100',
    'district': 'Chaimite',
    'suburb': 'Maquinino',
    'street': 'Rua Pedro Álvares Cabral',
    'lon': 34.83994428202128,
    'lat': -19.83683565,
    'formatted': 'Hotel Savoy, Rua Pedro Álvares Cabral, 2100 Beira, Mozambique',
    'address_line1': 'Hotel Savoy',
    'address_line2': 'Rua Pedro Álvares Cabral, 2100 Beira, Mozambique',
    'categories': ['accommodation',
     'accommodation.hotel',
     'building',
     'building.accommodation'],
    'details': [],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database Licence',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': 'Hotel 

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {
          "categories": "accommodation.hotel",
          "limit": 20,
           "filter": "",
           "bias": "",
#            "radius":radius,
           "apiKey": geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude=row["Lng"]
    latitude=row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    # Convert the API response to JSON format
    name_address = requests.get(base_url,params=params).json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
beira - nearest hotel: Hotel Savoy
timbuktu - nearest hotel: Hotel La Maison
salalah - nearest hotel: Muscat International Hotel
shani - nearest hotel: No hotel found
stephenville - nearest hotel: No hotel found
el granada - nearest hotel: Beach House
dhidhdhoo - nearest hotel: Cinnamon Island
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
kontcha - nearest hotel: No hotel found
atar - nearest hotel: فندق سكليل
maintirano - nearest hotel: Hotel Safari
graaff-reinet - nearest hotel: Profcon Resort
long beach - nearest hotel: Westin
laguna - nearest hotel: Holiday Inn Express & Suites
taoudenni - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
34,beira,MZ,-19.8436,34.8389,83,Hotel Savoy
77,timbuktu,ML,16.7735,-3.0074,12,Hotel La Maison
144,salalah,OM,17.0151,54.0924,69,Muscat International Hotel
146,shani,NG,10.2162,12.0620,16,No hotel found
181,stephenville,US,32.2207,-98.2023,32,No hotel found
239,el granada,US,37.5027,-122.4694,70,Beach House
261,dhidhdhoo,MV,6.8833,73.1000,66,Cinnamon Island
298,lompoc,US,34.6391,-120.4579,42,Embassy Suites by Hilton Lompoc Central Coast
382,kontcha,CM,7.9667,12.2333,20,No hotel found
414,atar,MR,20.5169,-13.0499,11,فندق سكليل


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size="Humidity",
    color="City",
    frame_width = 700,
    frame_height = 500,
    hover = True,
    hover_cols=["Hotel Name","Country"]
)

 
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)